In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from mpl_toolkits.mplot3d import axes3d
from scipy.interpolate import griddata, interp2d

from pathlib import Path

ncolors = 256
color_array = plt.get_cmap('gist_rainbow')(range(ncolors))
color_array[:,-1] = np.linspace(1.0,0.0,ncolors)
map_object = colors.LinearSegmentedColormap.from_list(name='rainbow_alpha',colors=color_array)
plt.register_cmap(cmap=map_object)


In [ ]:
paths = []
results = {}

# paths.append(Path("../../results/distances/20220807-185201.json")) # 3000 normal phi

# paths.append(Path("../../results/distances/20220807-201939.json")) # 500 normal phi
# paths.append(Path("../../results/distances/20220807-215412.json")) # 500 big phi

# paths.append(Path("../../results/distances/20220808-105411.json")) # 1500 small phi
# paths.append(Path("../../results/distances/20220807-164740.json")) # 1500 normal phi
# paths.append(Path("../../results/distances/20220808-100633.json")) # 1500 big phi
# paths.append(Path("../../results/distances/20220809-142931.json")) # 1500 many phi


# paths.append(Path("../../results/distances/20220808-133109.json")) # 1500 T_half
# paths.append(Path("../../results/distances/20220808-134307.json")) # 1500 T_half long

# paths.append(Path("../../results/distances/20220808-142650.json")) # 1500 mix phi/T_half
# paths.append(Path("../../results/distances/20220808-171339.json")) # 1500 mix phi/T_half
# paths.append(Path("../../results/distances/20220808-184217.json")) # 1500 mix phi/T_half
# paths.append(Path("../../results/distances/20220808-203450.json"))
# paths.append(Path("../../results/distances/20220808-222412.json"))
# paths.append(Path("../../results/distances/20220808-231818.json"))
# paths.append(Path("../../results/distances/20220809-112418.json"))

# idk
# paths.append(Path("../../results/distances/20220816-150832.json")) #non?
# paths.append(Path("../../results/distances/20220816-150919.json")) #

# epsilon vs length
# paths.append(Path("../../results/distances/20220816-155658.json")) # non 1it
# paths.append(Path("../../results/distances/20220816-155951.json")) # cheat 1it

# paths.append(Path("../../results/distances/20220817-090440.json")) # both 20it each
# paths.append(Path("../../results/distances/20220817-092428.json")) # non cheat 20it
# paths.append(Path("../../results/distances/20220817-093428.json")) # cheat 20it

# # phi vs ctot
# paths.append(Path("../../results/distances/20220818-113149.json")) # cheat
# paths.append(Path("../../results/distances/20220818-151420.json")) # cheat missing
# paths.append(Path("../../results/distances/20220819-100707.json")) # cheat extended
# paths.append(Path("../../results/distances/20220819-212354.json")) # cheat extended 2

paths.append(Path("../../results/distances/20220818-133126.json")) # non cheat 220819-101804
paths.append(Path("../../results/distances/20220819-101804.json"))
paths.append(Path("../../results/distances/20220819-214118.json"))

# # phi vs beta vs epsilon * l
# paths.append(Path("../../results/distances/20220821-212539.json"))


for path in paths:
    with path.open() as f:
        result = json.load(f)
        results.update(result)

In [ ]:
parameter = "phi"
data = [(v["parameters"]["cx"]["params"][parameter],v["mean_min_distance"]) for v in results.values()]
data = sorted(data)

visited_data = set()

out = []
for a, b in data:
    if not a in visited_data:
        visited_data.add(a)
        out.append((a, b))
        
data = out

min_x,min_y = zip(min(data, key=lambda tup: tup[1]))
x,y = zip(*data)

plt.figure(figsize=(10,10))
plt.plot(x,y)
plt.scatter(x,y)
plt.plot(min_x, min_y, marker="o", markersize=10, color="g",
            label=f"minimum {min_y[0]} at {parameter}={min_x[0]}")
plt.xscale("log")
plt.xlabel(parameter)
plt.ylabel("Mean closest distance")
plt.legend()


In [ ]:
# 3D plots
parameter_1 = "c_tot"
parameter_2 = "phi"
data = [(v["parameters"]["cx"]["params"][parameter_1],v["parameters"]["cx"]["params"][parameter_2],v["mean_min_distance"]) for v in results.values()]


data = sorted(data)
data = [tup for tup in data if tup[0] > 1e-2]
# print(len(data))

minimum = zip(min(data, key=lambda tup: tup[2]))
print(*minimum)

x,y,z = zip(*data)
# x = np.log10(x)
y = np.log10(y)
grid_x,grid_y = np.mgrid[min(x):max(x):100j, min(y):max(y):100j]
grid_z = griddata((x,y),z,(grid_x,grid_y),method='linear',rescale=True) # method can be nearest, linear or cubic

plt.figure(figsize=(9,9))

ax = plt.axes(projection='3d', computed_zorder=False)
ax.plot_surface(grid_x,grid_y,grid_z,cmap=cm.Spectral)
ax.view_init(elev=90, azim=0)

plt.xlabel(f"$log(\\{parameter_1}\cdot l)$")
plt.ylabel("$log(\\phi)$")
# plt.rcParams['font.size'] = 11

color_map = cm.ScalarMappable(cmap=cm.Spectral)
color_map.set_array(z)

# ax.scatter(*(x,y,z))
# ax.scatter(*minimum, alpha=1, s=100, marker="o")
plt.colorbar(color_map,fraction=0.03, pad=0.02)
plt.show()

In [ ]:
### 4D plots ###
parameter_1 = "epsilon"
parameter_2 = "phi"
parameter_3 = "beta"
data = [(v["parameters"]["cx"]["params"][parameter_1],v["parameters"]["cx"]["params"][parameter_2],v["parameters"]["cx"]["params"][parameter_3],v["mean_min_distance"]) for v in results.values()]

data_copy = data
dist_lim = 250
data = [tup for tup in data if tup[3] < dist_lim]

minimum = zip(min(data, key=lambda tup: tup[3]))
print(*minimum)

# Datapoints
xc,yc,zc,hc = zip(*data_copy)
x,y,z,h = zip(*data)
x = np.log10(x)
y = np.log10(y)

# Figure
fig = plt.figure(figsize=(9,9))
ax = plt.axes(projection='3d')
ax.view_init(elev=45, azim=45)

# Colormap
color_map = cm.ScalarMappable(cmap="rainbow_alpha")
color_map.set_array(hc)

sc = ax.scatter(x,y,z,c=h,s=100,marker='h',cmap=color_map.get_cmap(), norm=colors.Normalize(min(hc),max((hc))))

# Labels
ax.set_xlabel(f"$log(\\{parameter_1}\cdot l)$")
ax.set_ylabel(f"$log(\\{parameter_2})$")
ax.set_zlabel(f"$\\{parameter_3}$")

#Limits
ax.set_ylim(-5.5,-0.5)
ax.set_xlim(-2.5,2.5)
ax.set_zlim(-0.01,0.51)

# plt.rcParams['font.size'] = 11
# plt.colorbar(sc,cmap=color_map,norm=norm,boundaries=bounds)
plt.colorbar(color_map, fraction=0.03, pad=0.04)
plt.show()